<a href="https://colab.research.google.com/github/rvraghvender/DeepLearningProjects/blob/main/ConvolutionNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolution Neural Networks


In [1]:
# import relevant packages
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0)  # size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

# Outline


*   Convolution functions
    *   Zero Padding
    *   Convolve window
    *   Convolution forward
    *   Convolution backward

*   Pooling functions
    *   Pooling forward
    *   Create mask
    *   Distribute value
    *   Pooling backward



## 3.1 Zero padding

Zero padding adds zeros around the border of the image

Main benefits are:  

*   It allows to use a CONV layer without necessarily shrinking the height and width of the volumes. This is important for building deeper networks, since otherwise height/width would shrink as you go to deeper layers. An important case is the 'same' convolution, in which the height/width is exactly preserved after operation on a layer.

*   It helps to keep the information at the border of an image. Without padding, very few values at the next layer would be affected by pixels at  the edges of an image.

In [3]:
# zero_pad

def zero_pad(X, pad):
    '''
    Pad with zeros all image of the dataset X. 
    The padding is applied to the height and width of an image.

    Args:
    X -- numpy array of shape (m, n_H, n_W, n_c) representing a batch of m images.
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions

    Returs:
    X_pad -- padded image of shape (m, n_H  + 2 * pad, n_W + 2 * pad, n_C )
    '''

    
    # To pad a array "a" of shape (5,5,5,5,5) with pad = 1 for the 2nd dimension,
    # pad = 3 for the 4th dimesion and pad = 0 for the rest, it goes as
    # a = np.pad(a, ((0,0), (1,1), (0,0), (3,3), (0,0)), mode='constant', constant_values=(0,0)) 

    X_pad = np.pad(X, ((0,0), (pad, pad),  (pad,pad) , (0,0)))

    return X_pad

## 3.2 Single step of Convolution

Implement a single step of convolution, in which filter is applied to single position of the input.

*    Takes an input volume
*    Applies a filter at every position of the input
*    Outputs another volume (usually of different size)

Stride : amount to move the window each time slide is performed

In [5]:
# conv_single_step

def conv_single_step(a_slice_prev, W, b):
    '''
    Apply one filter defined by parameters W on a single slice (a_slice_prev) 
    of the output activation of the pervious layer.

    Args:
    a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
    W -- weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
    b -- bias parameters contained in a window - matrix of shape (1, 1, 1)

    Returns:
    Z -- a scalar value, the result of convolving the sliding window (W, b) on 
    a slice x of the input data
    '''

    # Element-wise product between a_slice_prev and W. Do not add the bias yet.
    s = np.multiply(a_slice_prev, W)

    # sum over all entries of the volume s.
    Z = np.sum(s)

    # Add bias b to Z. Cast b to float() so that Z results in a scalar value.
    b = np.squeeze(b)

    Z = Z + b

    return Z

## 3.3 Convolution Neural Networks - Forward Pass

In forward pass, we have to take many filters and convolve them on the input. Each 'convolution' gives you a 2D matrix output. Thereafter, stack these outputs to get a 3D volume.

Implement the function below to convolve the filters `W` on an input activation `A_prev`.  
This function takes the following inputs:
* `A_prev`, the activations output by the previous layer (for a batch of m inputs); 
* Weights are denoted by `W`.  The filter window size is `f` by `f`.
* The bias vector is `b`, where each filter has its own (single) bias.       


1.   To select a 2x2 slice at the upper left corner of a matrix "a_prev" (shape (5,5,3)), you would do:
```python
a_slice_prev = a_prev[0:2,0:2,:]
```
Notice how this gives a 3D slice that has height 2, width 2, and depth 3.  Depth is the number of channels.  
This will be useful when you will define `a_slice_prev` below, using the `start/end` indexes you will define.

2.   To define a_slice you will need to first define its corners `vert_start`, `vert_end`, `horiz_start` and `horiz_end`. This figure may be helpful for you to find out how each of the corners can be defined using h, w, f and s in the code below.


**Reminder**:
    
The formulas relating the output shape of the convolution to the input shape are:
    
$$n_H = \Bigl\lfloor \frac{n_{H_{prev}} - f + 2 \times pad}{stride} \Bigr\rfloor +1$$
$$n_W = \Bigl\lfloor \frac{n_{W_{prev}} - f + 2 \times pad}{stride} \Bigr\rfloor +1$$
$$n_C = \text{number of filters used in the convolution}$$
    

In [6]:
# conv_forward
def conv_forward(A_prev, W, b, hyperparameters):
    '''
    Implement the forward propagation for a convolution function

    Args:
    A_prev -- output activation of the previous layer
              numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- weights, numpy array of shape (f, f, n_C_prev, n_C)
    b -- biases, numpy array of shape (1, 1, 1, n_C)
    hyperparameters -- dictionary containing 'stride' and 'pad'

    Returns:
    z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function
    '''

    # Retrieve dimensions from A_prev's shape 
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape

    # Retrieve dimensions from W's shape
    (f, f, n_C_prev, n_C) = W.shape

    # Retrieve information from "hyperparameters"
    stride = hyperparameters['stride']
    pad = hyperparameters['pad']

    # Compute the dimesions of the CONV output volume
    n_H = int((n_H_prev + 2*pad -f) / stride) + 1
    n_W = int((n_W_prev + 2*pad -f) / stride) + 1

    # Initialize the output volume Z with zeros
    Z = np.zeros((m, n_H, n_W, n_C))

    # Create A_prev_pad by padding A_prev
    A_prev_pad = zero_pad(A_prev, pad)

    for i in range(m):
        a_prev_pad = A_prev_pad[i]

        for h in range(n_H):
            vert_start = stride * h
            vert_end = vert_start  + filter

            for w in range(n_W):
                horiz_start = stride * w
                horiz_end = horiz_start + f

                for c in range(n_C):
                    a_slice_prev = a_prev_pad[vert_start : vert_end, horiz_start : horiz_end, :]

                    weights = W[:, :, :, c]
                    biases = b[:, :, :, c]

                    z[i, h, w, c] = conv_single_step(a_slice_prev, weights, biases)

    cache = (A_prev, W, b, hyperparameters)

    return Z, cache

## 4 - Pooling layer
 The pooling (POOL) layer reduces the height and width of the input. It helps reduce computation, as well as helps make feature detectors more invariant to its position in the input. The two types of pooling layers are:
 *    Max-pooling layer: slides an (f, f) window over the input and stores the max value of the window in the output.
 *    Average-pooling layer: slides an (f, f) window over the input and stores the average value of the window in the output.


 These pooling layers have no parameters for backpropagation to train. However, they have hyperparameters such as the window size f. This specifies the height and width of the f x f window to compute a max or average over.

 These pooling layers have no parameters for backpropagation to train. However, they have hyperparameters such as the window size $f$. This specifies the height and width of the $f \times f$ window you would compute a *max* or *average* over. 

<a name='4-1'></a>
### 4.1 - Forward Pooling
 Implement MAX-POOL and AVG-POOL, in the same function. 


Implement the forward pass of the pooling layer. Follow the hints in the comments below.

**Reminder**:
As there's no padding, the formulas binding the output shape of the pooling to the input shape is:

$$n_H = \Bigl\lfloor \frac{n_{H_{prev}} - f}{stride} \Bigr\rfloor +1$$

$$n_W = \Bigl\lfloor \frac{n_{W_{prev}} - f}{stride} \Bigr\rfloor +1$$

$$n_C = n_{C_{prev}}$$



In [7]:
def pool_forward(A_prev, hyperparameters, mode = 'max'):
    '''
    Implements the forward pass of the pooling layer

    Args:
    A_prev -- Input data, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    hyperparameters -- dictionary containing 'f' and 'stride'
    mode -- the pooling mode to use, defined ad ('max' or 'average')

    Returns:
    A -- output of the pool layer, a numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache used in the backward pass of the pooling layer, contains the 
                input and hyperparameters 
    '''

    # Retrieve dimensions from the input shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape


    # Retrieve hyperparameters of the output
    n_H = int(1 + (n_H_prev - f ) / stride)
    n_W = int(1 + (n_W_prev - f ) / stride)
    n_C = n_C_prev
    

    # Initialize output matrix A
    A = np.zeros((m, n_H, n_W, n_C))


    for i in range(m):                  # loop over the training examples
        a_prev_slice = A_prev[i]

        for h in range(n_H):            # loop on the vertical axis of the output volume
            
            vert_start = stride * h 
            vert_end = vert_start + f
            
            for w in range(n_W):        # loop on the horizontal axis of the output volume
                
                horiz_start = stride * w
                horiz_end = horiz_start + f
                
                for c in range (n_C):            # loop over the channels of the output volume
                    
                    a_slice_prev = a_prev_slice[vert_start:vert_end,horiz_start:horiz_end,c]
                    
                    # Compute the pooling operation on the slice. 
                    if mode == "max":
                        A[i, h, w, c] = np.max(a_slice_prev)
                    elif mode == "average":
                        A[i, h, w, c] = np.mean(a_slice_prev)
                    else:
                        print(mode+ "-type pooling layer NOT Defined")    
    
    cache = (A_prev, hyperparameters)

    # Making sure output shape is correct
    assert(A.shape == (m, n_H, n_W, n_C))
    
    return A, cache
